# 测试Attention block

In [ ]:
import numpy as np
np.random.seed(1)
import tensorflow as tf
tf.set_random_seed(1234)
from keras.layers import *
from keras.models import *
from keras.optimizers import Adam
from keras.regularizers import l1, l2
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers
import scipy.sparse as sp
import keras.backend as K

def attention_3d_block(inputs):
    input_dim = int(inputs.shape[2])
    a = Permute((1, 2))(inputs)
    a = Dense(32, activation='softmax')(a)
    a_probs = Permute((1, 2))(a)
    output_attention_mul = Multiply()([inputs, a_probs])
    return output_attention_mul

# Keras 基本操作
- 转换tensor: `tf.convert_to_tensor`
- 打印tensor值: `keras.backend.get_value`

In [ ]:
bs = 3
h = 2
w = 2
channels = 6
a = np.random.randint(low=0, high=10, size=(bs,h,w,channels)).astype(float) # batch_size, height, weight, channels
y = np.random.randint(low=0, high=2, size=bs)

In [ ]:
a

In [ ]:
inputs = Input(shape=(h,w,channels))
x = Reshape((-1,channels))(inputs)
x = Permute((1,2))(x)
x = Softmax()(x)
output1 = (x)

model = Model(inputs=inputs, outputs=output1)

In [ ]:
b = model(tf.convert_to_tensor(a, dtype=tf.float32))

In [ ]:
out = K.get_value(b)

In [ ]:
out[0].sum(axis=1)

In [ ]:
print(out.shape)
print(out)


# Pytorch PHIAF

In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import numpy as np    
import random 
random.seed(1)
from metrics import scores
from sklearn.model_selection import KFold
import math
from sklearn.model_selection import StratifiedKFold

from CNNModel import *
from torch.utils.data import Dataset, DataLoader
from tensorboardX import 

EPOCHS=300
INIT_LR=1e-3

class ImageDataset(Dataset):
    def __init__(self, dna, pro, y):
        self.dna = dna
        self.pro = pro
        self.y = y
        
    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.dna[idx], self.pro[idx], self.y[idx]

def reshapes(X_en_tra,X_pr_tra,X_en_val,X_pr_val):
    sq=int(math.sqrt(X_en_tra.shape[1]))
    if pow(sq,2)==X_en_tra.shape[1]:
        X_en_tra2=X_en_tra.reshape((-1,sq,sq))
        X_pr_tra2=X_pr_tra.reshape((-1,sq,sq))
        X_en_val2=X_en_val.reshape((-1,sq,sq))
        X_pr_val2=X_pr_val.reshape((-1,sq,sq))
    else:
        X_en_tra2=np.concatenate((X_en_tra,np.zeros((X_en_tra.shape[0],int(pow(sq+1,2)-X_en_tra.shape[1])))),
                                  axis=1).reshape((-1,sq+1,sq+1))
        X_pr_tra2=np.concatenate((X_pr_tra,np.zeros((X_pr_tra.shape[0],int(pow(sq+1,2)-X_pr_tra.shape[1])))),
                                  axis=1).reshape((-1,sq+1,sq+1))
        X_en_val2=np.concatenate((X_en_val,np.zeros((X_en_val.shape[0],int(pow(sq+1,2)-X_en_val.shape[1])))),
                                  axis=1).reshape((-1,sq+1,sq+1))
        X_pr_val2=np.concatenate((X_pr_val,np.zeros((X_pr_val.shape[0],int(pow(sq+1,2)-X_pr_val.shape[1])))),
                                  axis=1).reshape((-1,sq+1,sq+1))
    return X_en_tra2, X_pr_tra2, X_en_val2, X_pr_val2

def obtainfeatures(data,file_path1,file_path2,strs):
    phage_features=[]
    host_features=[]
    labels=[]
    for i in data:
        phage_features.append(np.loadtxt(file_path1+i[0]+strs).tolist())
        host_features.append(np.loadtxt(file_path2+i[1].split('.')[0]+strs).tolist())
        labels.append(i[-1])
    return np.array(phage_features), np.array(host_features), np.array(labels)

def obtain_neg(X_tra,X_val):    
    X_tra_pos=[mm for mm in X_tra if mm[2]==1]
    X_neg=[]
    training_neg=[]
    phage=list(set([mm[0]for mm in X_tra_pos]))
    host=list(set([mm[1]for mm in X_tra_pos]))
    for p in phage:
        for h in host:
            if str(p)+','+str(h) in X_neg:
                continue
            else:
                training_neg.append([p,h,0])
    return random.sample(training_neg,len(X_tra_pos))

result_all=[]
pred_all=[]
test_y_all=[]

data1=pd.read_csv('../data/data_pos_neg.txt',header=None,sep=',')
data1=data1[data1[2]==1]
allinter=[str(data1.loc[i,0])+','+str(data1.loc[i,1]) for i in data1.index]
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
training=pd.read_csv('../data/data_pos_neg.txt',header=None,sep=',').values.tolist()
training_tar = [tt[2] for tt in training]
for train_index, test_index in skf.split(training, training_tar): 
    ###obtain data
    X_tra=[training[ii] for ii in train_index if training[ii][2] == 1]
    X_val=[training[ii] for ii in test_index]
    neg_select=obtain_neg(X_tra,X_val)  ##add extra negative samples
    X_phage_tra_dna,X_host_tra_dna,y_tra=obtainfeatures(X_tra+neg_select,'../data/phage_dna_norm_features/','../data/host_dna_norm_features/','.txt')
    X_phage_val_dna,X_host_val_dna,y_val=obtainfeatures(X_val,'../data/phage_dna_norm_features/','../data/host_dna_norm_features/','.txt')
    X_phage_tra_pro,X_host_tra_pro,_=obtainfeatures(X_tra+neg_select,'../data/phage_protein_normfeatures/','../data/host_protein_normfeatures/','.txt')
    X_phage_val_pro,X_host_val_pro,_=obtainfeatures(X_val,'../data/phage_protein_normfeatures/','../data/host_protein_normfeatures/','.txt')
    
    # Reshapes to image -> [bs, c, h, w]
    X_phage_tra_dna3,X_host_tra_dna3,X_phage_val_dna3,X_host_val_dna3=reshapes(X_phage_tra_dna,X_host_tra_dna,X_phage_val_dna,X_host_val_dna)
    X_phage_tra_pro3,X_host_tra_pro3,X_phage_val_pro3,X_host_val_pro3=reshapes(X_phage_tra_pro,X_host_tra_pro,X_phage_val_pro,X_host_val_pro)
    X_dna=np.array([X_phage_tra_dna3,X_host_tra_dna3]).transpose(1,0,2,3)
    X_pro=np.array([X_phage_tra_pro3,X_host_tra_pro3]).transpose(1,0,2,3)
    
    # Train
    train_dataset = ImageDataset(torch.tensor(X_dna), torch.tensor(X_pro), torch.tensor(y_tra))
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    
    





In [1]:
from CNNModel import *

in_channels = 2
out_channels = 32
kernel_size = 3
dna_pool_size = 3
pro_pool_size = 2
net = PHIAFModel(in_channels, out_channels, kernel_size, dna_pool_size, pro_pool_size)

/home/misaki/miniconda3/envs/ups/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
X_dna = torch.rand(size=(4,2,19,19))
X_pro = torch.rand(size=(4,2,13,13))
out = net(X_dna, X_pro)

In [3]:
X_dna.dtype

torch.float32